## Usage

In [ ]:
import numpy as np
import pandas as pd
from copsens import CopSens

# Generate synthetic data
N = 1000
T = np.random.normal(0, 1, (N, 10)) # 10 Treatments
U = np.random.normal(0, 1, (N, 3))  # 3 Unobserved Confounders
# Y depends on T and U
y = T[:, 0] * 0.5 + U[:, 0] * 1.0 + np.random.normal(0, 0.5, N)

# Initialize Estimator
# Uses PCA by default to infer latent confounders
cs = CopSens(model_type='gaussian', n_components=3)

# Fit models
cs.fit(T, y)

# Define treatments for contrast (Treatment 0: 1 vs 0)
t1 = T.copy()
t1[:, 0] = 1
t2 = T.copy()
t2[:, 0] = 0

# Perform Calibration
results = cs.calibrate(t1, t2, calitype='worstcase', R2=[0.1, 0.5, 1.0])

print(results['est_df'].head())
print("Robustness Values:", results['rv'][:5])

In [ ]:
from copsens import CopSens
import torch.nn as nn

# Initialize with PyTorch backend
# 'latent_model' can be an instantiated nn.Module or just 'vae' (uses internal SimpleVAE)
cs_torch = CopSens(model_type='gaussian', latent_model='vae', n_components=3)

cs_torch.fit(T, y)

# Multivariate Calibration (minimizing L2 norm of effects)
results = cs_torch.calibrate(t1, t2, calitype='multicali', R2_constr=0.5)

print("Optimized Gamma:", results['gamma'])
print(results['est_df'].head())

In [ ]:
# Binary outcome
y_bin = (y > 0).astype(int)

cs_bin = CopSens(model_type='binary', n_components=3)
cs_bin.fit(T, y_bin)

# Naive estimates (Probabilities)
naive_probs = cs_bin.predict(T)

# Calibrate with specific gamma direction
gamma_guess = np.array([1.0, -0.5, 0.0])
res_bin = cs_bin.calibrate(t1, t2, calitype='null', gamma=gamma_guess, R2=[0.2])

print(res_bin['est_df'].head())

In [ ]:
# ... training code ...
cs_torch.fit(T, y)

# 1. Specify query point(s) x* (can be a batch)
x_star = T[0:5] # First 5 images

# 2. Get bounds for specific sensitivity parameters R2
# Returns a dictionary keyed by R2
bounds = cs_torch.predict_bounds(x_star, R2=[0.1, 0.5, 1.0])

# 3. Access results
print("--- Bounds for R^2 = 0.5 ---")
print(bounds[0.5])

***